In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Read the CSV file
py_df = pd.read_csv('code_generation_results_gpt-4o-mini_py.csv')
js_df = pd.read_csv('code_generation_results_gpt-4o-mini_js.csv')

In [3]:
def print_stats(df):
    # Initialize prior success count
    prior_success = len(df[df['attempt'] == 0])

    prior_func_sec = 0

    # Iterate through attempts and calculate success rates
    for attempt in range(3):
        attempt_df = df[df['attempt'] == attempt]

        tot = len(attempt_df) + prior_func_sec
        compiled = attempt_df['compiles'].sum() + prior_func_sec
        functional = attempt_df['functional'].sum() + prior_func_sec
        secure = attempt_df['secure'].sum() + prior_func_sec
        func_secure = attempt_df['func_secure'].sum() + prior_func_sec

        print(f"\nUp to attempt {attempt + 1}:")
        print(f"Compiled: {compiled/tot * 100}")
        print(f"Functional: {functional/tot * 100}")
        print(f"Secure: {secure/tot * 100}")
        print(f"Func-Sec: {func_secure/tot * 100}")

        prior_func_sec = func_secure

In [4]:
print_stats(py_df)


Up to attempt 1:
Compiled: 95.83333333333334
Functional: 83.33333333333334
Secure: 75.0
Func-Sec: 75.0

Up to attempt 2:
Compiled: 100.0
Functional: 91.66666666666666
Secure: 91.66666666666666
Func-Sec: 87.5

Up to attempt 3:
Compiled: 100.0
Functional: 91.66666666666666
Secure: 91.66666666666666
Func-Sec: 87.5


In [5]:
print_stats(js_df)


Up to attempt 1:
Compiled: 100.0
Functional: 45.45454545454545
Secure: 50.0
Func-Sec: 40.909090909090914

Up to attempt 2:
Compiled: 100.0
Functional: 54.54545454545454
Secure: 54.54545454545454
Func-Sec: 45.45454545454545

Up to attempt 3:
Compiled: 100.0
Functional: 68.18181818181817
Secure: 50.0
Func-Sec: 50.0
